In [1]:
import numpy as np
# データの図を出力するライブラリ
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

# 環境設定
import sys
sys.path.append('models')

# 実際の機械学習ライブラリ
import tensorflow as tf

In [2]:
# Google Driveからデータを引っ張ってくる
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/MyDrive/shibaki analysis/AE-FILD BAL patient_data.xlsx' patient_data.xlsx

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# openpyxlというエンジンを使用して、ExcelファイルをPandasのデータフレームとして読みこむ
#!pip install openpyxl
import pandas as pd
AE_FILD_df = pd.read_excel('patient_data.xlsx', engine='openpyxl') 

In [4]:
AE_FILD_df.head()

,No,ID,90day mortality\nsurvival:0\ndeath:1,sex\nmale:0\nfemale:1,age,smoking status\nNever:0\nEx(quit smoking over a year ago):1\nCurrent:2,Brinkman Index,emphysema\n(-):0\n(+):1,Baseline ILD\nUIP:0\nprobable UIP:1\nIndeterminate for UIP:2\nAlternative diagnosis:3,Collagen disease\n(-):0\n(+):1,IPAF\n(-):0\n(+):1,P/F ratio,LDH,CRP,KL-6,\nBAL(neutro %),\nBAL(Lymph %),\nBAL(Eos %),\nBAL(M %),\nBAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,K003,558277,0,0,78,1,1600,0,1,0,0,291.666667,345,2.35,1599,41,38,3.0,13,3.5,664.773204,13313.791597,62.176805,40.438809,9.765625,4.450383,NaN,10.963133,3.200000,10.836599,3.200000,3.200000,33.265989,3.200000,3.200000,954.746558,3.200000,34.734508,3.2,81.530691,3.2,3.2,3.2,12.667984,3.2,395.799401,3.379747,3.2,3.2,3.2,3.2,14.451335,3.20000,23.016166,3.2,313.439925,2160.940947,1009.878650,28.906316,30.940577,4.483265,3.2,3.2,4.221958
1,K022,1848827,0,1,72,0,0,0,2,0,0,264.571429,277,5.65,1620,18,21,0.0,61,3.9,561.092206,12107.253258,83.356309,64.460473,9.765625,6.932559,NaN,15.963448,3.200000,53.038298,3.200000,4.635806,75.733522,6.710735,10.849171,2849.703534,9.801179,46.135602,3.2,60.021260,3.2,3.2,3.2,16.925359,3.2,1149.151761,6.373706,3.2,3.2,3.2,3.2,26.603585,3.20000,60.512609,3.2,325.398968,15647.940247,1943.663852,25.105166,25.342591,24.074274,3.2,3.2,2.000000
2,K024,1902349,0,0,86,0,0,0,2,0,0,250.000000,269,6.50,1073,2,81,4.0,12,2.5,57.921345,6617.937597,17.636285,16.788346,9.765625,3.802534,NaN,5.767947,3.200000,32.800163,8.791588,3.200000,62.765297,3.867359,3.200000,433.780612,3.200000,4.018996,3.2,14.035815,3.2,3.2,3.2,23.676270,3.2,67.058080,3.200000,3.2,3.2,3.2,3.2,12.288175,3.20000,3.200000,3.2,35.723941,2807.826476,243.983235,5.912140,23.698467,8.748703,3.2,3.2,2.000000
3,K043,3165531,0,0,58,1,500,0,2,0,0,200.000000,538,11.26,2392,13,47,14.0,25,4.3,292.713992,8254.956691,12.391890,40.705676,9.765625,3.802534,NaN,9.398526,3.200000,20.027641,3.200000,3.200000,40.928415,4.625713,3.200000,416.843721,3.200000,21.447348,NaN,36.754655,3.2,3.2,3.2,4.925806,3.2,205.470611,3.200000,3.2,3.2,3.2,3.2,11.174294,3.20000,24.517336,3.2,87.506323,3908.900383,637.958551,3.518690,14.870714,NaN,3.2,3.2,2.000000
4,K045,3253683,0,0,76,1,800,0,3,0,0,380.952381,372,6.43,1555,9,37,36.0,7,2.3,645.580358,27110.719567,69.344563,92.977880,9.765625,NaN,NaN,36.132460,3.992006,19.948654,9.554105,3.200000,16.046589,3.200000,3.200000,252.724913,38.574521,40.631309,3.2,946.005892,3.2,3.2,3.2,6.572539,3.2,1089.321448,3.200000,3.2,3.2,3.2,3.2,7.745387,33.27508,240.737287,3.2,179.661551,2575.467315,3495.474916,31.821785,100.036875,21.233261,3.2,3.2,2.000000


In [5]:
# ヘルパーファンクションでテキスト内の"\n"をスペースに変換する
def clean_header(text):
  text_n_newline = text.replace("\n", " ")
  return text_n_newline

In [6]:
# ヘッダーコラムのテキストをリストとして変換する
AE_FILD_header_list = AE_FILD_df.columns.values.tolist()
AE_FILD_header_list

['No',
 'ID',
 '90day mortality\nsurvival:0\ndeath:1',
 'sex\nmale:0\nfemale:1',
 'age ',
 'smoking status\nNever:0\nEx(quit smoking over a year ago):1\nCurrent:2',
 'Brinkman Index',
 'emphysema\n(-):0\n(+):1',
 'Baseline ILD\nUIP:0\nprobable UIP:1\nIndeterminate for UIP:2\nAlternative diagnosis:3',
 'Collagen disease\n(-):0\n(+):1',
 'IPAF\n(-):0\n(+):1',
 'P/F ratio',
 'LDH',
 'CRP',
 'KL-6',
 '\nBAL(neutro %)',
 '\nBAL(Lymph %)',
 '\nBAL(Eos %)',
 '\nBAL(M %)',
 '\nBAL(CD4/8)',
 'MMP-1',
 'MMP-7',
 'TGFB1',
 'TGFB2',
 'TGFB3',
 'EGF',
 'FGF-2',
 'EOTAXIN',
 'TGF-a',
 'G-CSF',
 'Flt-3L',
 'GM-CSF',
 'FRACTALKINE',
 'IFN-a2',
 'IFN-g',
 'GRO',
 'IL-10',
 'MCP-3',
 'IL-12P40',
 'MDC',
 'IL-12P70',
 'IL-13',
 'IL-15',
 'sCD40L',
 'IL-17',
 'IL-1RA',
 'IL-1a',
 'IL-9',
 'IL-1b',
 'IL-2',
 'IL-3',
 'IL-4',
 'IL-5',
 'IL-6',
 'IL-7',
 'IL-8',
 'IP-10',
 'MCP-1',
 'MIP-1a',
 'MIP-1b',
 'TNFa',
 'TNFb',
 'VEGF',
 'IL-36β']

In [7]:
#　上記のヘッダーリストにclean_header()を応用する
AE_FILD_header_list_cleaned = [clean_header(x) for x in AE_FILD_header_list]
print(AE_FILD_header_list_cleaned)

['No', 'ID', '90day mortality survival:0 death:1', 'sex male:0 female:1', 'age ', 'smoking status Never:0 Ex(quit smoking over a year ago):1 Current:2', 'Brinkman Index', 'emphysema (-):0 (+):1', 'Baseline ILD UIP:0 probable UIP:1 Indeterminate for UIP:2 Alternative diagnosis:3', 'Collagen disease (-):0 (+):1', 'IPAF (-):0 (+):1', 'P/F ratio', 'LDH', 'CRP', 'KL-6', ' BAL(neutro %)', ' BAL(Lymph %)', ' BAL(Eos %)', ' BAL(M %)', ' BAL(CD4/8)', 'MMP-1', 'MMP-7', 'TGFB1', 'TGFB2', 'TGFB3', 'EGF', 'FGF-2', 'EOTAXIN', 'TGF-a', 'G-CSF', 'Flt-3L', 'GM-CSF', 'FRACTALKINE', 'IFN-a2', 'IFN-g', 'GRO', 'IL-10', 'MCP-3', 'IL-12P40', 'MDC', 'IL-12P70', 'IL-13', 'IL-15', 'sCD40L', 'IL-17', 'IL-1RA', 'IL-1a', 'IL-9', 'IL-1b', 'IL-2', 'IL-3', 'IL-4', 'IL-5', 'IL-6', 'IL-7', 'IL-8', 'IP-10', 'MCP-1', 'MIP-1a', 'MIP-1b', 'TNFa', 'TNFb', 'VEGF', 'IL-36β']


In [8]:
AE_FILD_df.columns = AE_FILD_header_list_cleaned
AE_FILD_df.head()

,No,ID,90day mortality survival:0 death:1,sex male:0 female:1,age,smoking status Never:0 Ex(quit smoking over a year ago):1 Current:2,Brinkman Index,emphysema (-):0 (+):1,Baseline ILD UIP:0 probable UIP:1 Indeterminate for UIP:2 Alternative diagnosis:3,Collagen disease (-):0 (+):1,IPAF (-):0 (+):1,P/F ratio,LDH,CRP,KL-6,BAL(neutro %),BAL(Lymph %),BAL(Eos %),BAL(M %),BAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,K003,558277,0,0,78,1,1600,0,1,0,0,291.666667,345,2.35,1599,41,38,3.0,13,3.5,664.773204,13313.791597,62.176805,40.438809,9.765625,4.450383,NaN,10.963133,3.200000,10.836599,3.200000,3.200000,33.265989,3.200000,3.200000,954.746558,3.200000,34.734508,3.2,81.530691,3.2,3.2,3.2,12.667984,3.2,395.799401,3.379747,3.2,3.2,3.2,3.2,14.451335,3.20000,23.016166,3.2,313.439925,2160.940947,1009.878650,28.906316,30.940577,4.483265,3.2,3.2,4.221958
1,K022,1848827,0,1,72,0,0,0,2,0,0,264.571429,277,5.65,1620,18,21,0.0,61,3.9,561.092206,12107.253258,83.356309,64.460473,9.765625,6.932559,NaN,15.963448,3.200000,53.038298,3.200000,4.635806,75.733522,6.710735,10.849171,2849.703534,9.801179,46.135602,3.2,60.021260,3.2,3.2,3.2,16.925359,3.2,1149.151761,6.373706,3.2,3.2,3.2,3.2,26.603585,3.20000,60.512609,3.2,325.398968,15647.940247,1943.663852,25.105166,25.342591,24.074274,3.2,3.2,2.000000
2,K024,1902349,0,0,86,0,0,0,2,0,0,250.000000,269,6.50,1073,2,81,4.0,12,2.5,57.921345,6617.937597,17.636285,16.788346,9.765625,3.802534,NaN,5.767947,3.200000,32.800163,8.791588,3.200000,62.765297,3.867359,3.200000,433.780612,3.200000,4.018996,3.2,14.035815,3.2,3.2,3.2,23.676270,3.2,67.058080,3.200000,3.2,3.2,3.2,3.2,12.288175,3.20000,3.200000,3.2,35.723941,2807.826476,243.983235,5.912140,23.698467,8.748703,3.2,3.2,2.000000
3,K043,3165531,0,0,58,1,500,0,2,0,0,200.000000,538,11.26,2392,13,47,14.0,25,4.3,292.713992,8254.956691,12.391890,40.705676,9.765625,3.802534,NaN,9.398526,3.200000,20.027641,3.200000,3.200000,40.928415,4.625713,3.200000,416.843721,3.200000,21.447348,NaN,36.754655,3.2,3.2,3.2,4.925806,3.2,205.470611,3.200000,3.2,3.2,3.2,3.2,11.174294,3.20000,24.517336,3.2,87.506323,3908.900383,637.958551,3.518690,14.870714,NaN,3.2,3.2,2.000000
4,K045,3253683,0,0,76,1,800,0,3,0,0,380.952381,372,6.43,1555,9,37,36.0,7,2.3,645.580358,27110.719567,69.344563,92.977880,9.765625,NaN,NaN,36.132460,3.992006,19.948654,9.554105,3.200000,16.046589,3.200000,3.200000,252.724913,38.574521,40.631309,3.2,946.005892,3.2,3.2,3.2,6.572539,3.2,1089.321448,3.200000,3.2,3.2,3.2,3.2,7.745387,33.27508,240.737287,3.2,179.661551,2575.467315,3495.474916,31.821785,100.036875,21.233261,3.2,3.2,2.000000


In [9]:
# まず機械学習に不必要なコラムをdrop()する
AE_FILD_df = AE_FILD_df.drop(columns=["No", "ID"])
AE_FILD_df.head()

,90day mortality survival:0 death:1,sex male:0 female:1,age,smoking status Never:0 Ex(quit smoking over a year ago):1 Current:2,Brinkman Index,emphysema (-):0 (+):1,Baseline ILD UIP:0 probable UIP:1 Indeterminate for UIP:2 Alternative diagnosis:3,Collagen disease (-):0 (+):1,IPAF (-):0 (+):1,P/F ratio,LDH,CRP,KL-6,BAL(neutro %),BAL(Lymph %),BAL(Eos %),BAL(M %),BAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,0,0,78,1,1600,0,1,0,0,291.666667,345,2.35,1599,41,38,3.0,13,3.5,664.773204,13313.791597,62.176805,40.438809,9.765625,4.450383,NaN,10.963133,3.200000,10.836599,3.200000,3.200000,33.265989,3.200000,3.200000,954.746558,3.200000,34.734508,3.2,81.530691,3.2,3.2,3.2,12.667984,3.2,395.799401,3.379747,3.2,3.2,3.2,3.2,14.451335,3.20000,23.016166,3.2,313.439925,2160.940947,1009.878650,28.906316,30.940577,4.483265,3.2,3.2,4.221958
1,0,1,72,0,0,0,2,0,0,264.571429,277,5.65,1620,18,21,0.0,61,3.9,561.092206,12107.253258,83.356309,64.460473,9.765625,6.932559,NaN,15.963448,3.200000,53.038298,3.200000,4.635806,75.733522,6.710735,10.849171,2849.703534,9.801179,46.135602,3.2,60.021260,3.2,3.2,3.2,16.925359,3.2,1149.151761,6.373706,3.2,3.2,3.2,3.2,26.603585,3.20000,60.512609,3.2,325.398968,15647.940247,1943.663852,25.105166,25.342591,24.074274,3.2,3.2,2.000000
2,0,0,86,0,0,0,2,0,0,250.000000,269,6.50,1073,2,81,4.0,12,2.5,57.921345,6617.937597,17.636285,16.788346,9.765625,3.802534,NaN,5.767947,3.200000,32.800163,8.791588,3.200000,62.765297,3.867359,3.200000,433.780612,3.200000,4.018996,3.2,14.035815,3.2,3.2,3.2,23.676270,3.2,67.058080,3.200000,3.2,3.2,3.2,3.2,12.288175,3.20000,3.200000,3.2,35.723941,2807.826476,243.983235,5.912140,23.698467,8.748703,3.2,3.2,2.000000
3,0,0,58,1,500,0,2,0,0,200.000000,538,11.26,2392,13,47,14.0,25,4.3,292.713992,8254.956691,12.391890,40.705676,9.765625,3.802534,NaN,9.398526,3.200000,20.027641,3.200000,3.200000,40.928415,4.625713,3.200000,416.843721,3.200000,21.447348,NaN,36.754655,3.2,3.2,3.2,4.925806,3.2,205.470611,3.200000,3.2,3.2,3.2,3.2,11.174294,3.20000,24.517336,3.2,87.506323,3908.900383,637.958551,3.518690,14.870714,NaN,3.2,3.2,2.000000
4,0,0,76,1,800,0,3,0,0,380.952381,372,6.43,1555,9,37,36.0,7,2.3,645.580358,27110.719567,69.344563,92.977880,9.765625,NaN,NaN,36.132460,3.992006,19.948654,9.554105,3.200000,16.046589,3.200000,3.200000,252.724913,38.574521,40.631309,3.2,946.005892,3.2,3.2,3.2,6.572539,3.2,1089.321448,3.200000,3.2,3.2,3.2,3.2,7.745387,33.27508,240.737287,3.2,179.661551,2575.467315,3495.474916,31.821785,100.036875,21.233261,3.2,3.2,2.000000


In [10]:
# データセット内のNaN値をどうするか (まずはinputer transformで試してみる)
from sklearn.impute import SimpleImputer
AE_FILD_df.replace('NaN', np.NaN, inplace=True) # データフレーム内の"NaN"をnumpyのNaNと置き換える
imp = SimpleImputer(missing_values=np.NaN, strategy="median") # sklearnのSimpleImputerを使い、np.NaNをnull値として認識させる。加、strategy="median"はnp.NaNをその行のNaNを中央値と置き換えるための指定
AE_FILD_df_idf = pd.DataFrame(imp.fit_transform(AE_FILD_df))
AE_FILD_df_idf.columns = AE_FILD_df.columns # SimpleImputerは数値しか作成できない為、元のデータセットの行名を新しいデータセットに加るコード
df_null_val = list(AE_FILD_df_idf.isna().sum()) #　データセット内にNaN値が無いかの確認
print(f"AE_FILD_df_idfデータフレーム内のNaN数：{df_null_val}")
AE_FILD_df_idf.head()

AE_FILD_df_idfデータフレーム内のNaN数：[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


,90day mortality survival:0 death:1,sex male:0 female:1,age,smoking status Never:0 Ex(quit smoking over a year ago):1 Current:2,Brinkman Index,emphysema (-):0 (+):1,Baseline ILD UIP:0 probable UIP:1 Indeterminate for UIP:2 Alternative diagnosis:3,Collagen disease (-):0 (+):1,IPAF (-):0 (+):1,P/F ratio,LDH,CRP,KL-6,BAL(neutro %),BAL(Lymph %),BAL(Eos %),BAL(M %),BAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,0.0,0.0,78.0,1.0,1600.0,0.0,1.0,0.0,0.0,291.666667,345.0,2.35,1599.0,41.0,38.0,3.0,13.0,3.5,664.773204,13313.791597,62.176805,40.438809,9.765625,4.450383,6.246947,10.963133,3.200000,10.836599,3.200000,3.200000,33.265989,3.200000,3.200000,954.746558,3.200000,34.734508,3.2,81.530691,3.2,3.2,3.2,12.667984,3.2,395.799401,3.379747,3.2,3.2,3.2,3.2,14.451335,3.20000,23.016166,3.2,313.439925,2160.940947,1009.878650,28.906316,30.940577,4.483265,3.2,3.2,4.221958
1,0.0,1.0,72.0,0.0,0.0,0.0,2.0,0.0,0.0,264.571429,277.0,5.65,1620.0,18.0,21.0,0.0,61.0,3.9,561.092206,12107.253258,83.356309,64.460473,9.765625,6.932559,6.246947,15.963448,3.200000,53.038298,3.200000,4.635806,75.733522,6.710735,10.849171,2849.703534,9.801179,46.135602,3.2,60.021260,3.2,3.2,3.2,16.925359,3.2,1149.151761,6.373706,3.2,3.2,3.2,3.2,26.603585,3.20000,60.512609,3.2,325.398968,15647.940247,1943.663852,25.105166,25.342591,24.074274,3.2,3.2,2.000000
2,0.0,0.0,86.0,0.0,0.0,0.0,2.0,0.0,0.0,250.000000,269.0,6.50,1073.0,2.0,81.0,4.0,12.0,2.5,57.921345,6617.937597,17.636285,16.788346,9.765625,3.802534,6.246947,5.767947,3.200000,32.800163,8.791588,3.200000,62.765297,3.867359,3.200000,433.780612,3.200000,4.018996,3.2,14.035815,3.2,3.2,3.2,23.676270,3.2,67.058080,3.200000,3.2,3.2,3.2,3.2,12.288175,3.20000,3.200000,3.2,35.723941,2807.826476,243.983235,5.912140,23.698467,8.748703,3.2,3.2,2.000000
3,0.0,0.0,58.0,1.0,500.0,0.0,2.0,0.0,0.0,200.000000,538.0,11.26,2392.0,13.0,47.0,14.0,25.0,4.3,292.713992,8254.956691,12.391890,40.705676,9.765625,3.802534,6.246947,9.398526,3.200000,20.027641,3.200000,3.200000,40.928415,4.625713,3.200000,416.843721,3.200000,21.447348,3.2,36.754655,3.2,3.2,3.2,4.925806,3.2,205.470611,3.200000,3.2,3.2,3.2,3.2,11.174294,3.20000,24.517336,3.2,87.506323,3908.900383,637.958551,3.518690,14.870714,3.792897,3.2,3.2,2.000000
4,0.0,0.0,76.0,1.0,800.0,0.0,3.0,0.0,0.0,380.952381,372.0,6.43,1555.0,9.0,37.0,36.0,7.0,2.3,645.580358,27110.719567,69.344563,92.977880,9.765625,4.350069,6.246947,36.132460,3.992006,19.948654,9.554105,3.200000,16.046589,3.200000,3.200000,252.724913,38.574521,40.631309,3.2,946.005892,3.2,3.2,3.2,6.572539,3.2,1089.321448,3.200000,3.2,3.2,3.2,3.2,7.745387,33.27508,240.737287,3.2,179.661551,2575.467315,3495.474916,31.821785,100.036875,21.233261,3.2,3.2,2.000000


In [11]:
# 今回の機械学習に使うライブラリーはsklearnのRF解析
#　sklearnが機械学習モデルを適応できるようにデータをnumpyのarrayに変換する
A = AE_FILD_df_idf.values #.valuesはnumpy arrayを出力する　
print(A)

[[ 0.          0.         78.         ...  3.2         3.2
   4.22195768]
 [ 0.          1.         72.         ...  3.2         3.2
   2.        ]
 [ 0.          0.         86.         ...  3.2         3.2
   2.        ]
 ...
 [ 0.          0.         74.         ...  3.2         3.2
   2.        ]
 [ 0.          1.         80.         ...  3.2         3.2
   2.        ]
 [ 0.          0.         71.         ...  3.2         3.2
   2.        ]]


In [12]:
#　後で、乗算と除算ができるように、すべての値を浮動小数点に変換する
A = A.astype('float64')
print(A)

[[ 0.          0.         78.         ...  3.2         3.2
   4.22195768]
 [ 0.          1.         72.         ...  3.2         3.2
   2.        ]
 [ 0.          0.         86.         ...  3.2         3.2
   2.        ]
 ...
 [ 0.          0.         74.         ...  3.2         3.2
   2.        ]
 [ 0.          1.         80.         ...  3.2         3.2
   2.        ]
 [ 0.          0.         71.         ...  3.2         3.2
   2.        ]]


In [13]:
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

# ガウス法に基づきまずデータセットの密度をnumpyで表すためbandwithの設定
# このbandwithの値に基づきデータ全体のパターンを機械学習モデルが予測しようとするため、機械学習モデルが勝手に最適bandwithを見つけてくれるものの、初期設定は重要
bandwith_params = {'bandwidth': np.arange(0.01,1,0.05)} # [0.01 から　１]の間でステップ0.05で値を作成
# モデルをスキャンして最適パラメーターを見つける機械学習モデル（模擬データを作成するのに最適）
grid_search = GridSearchCV(KernelDensity(), bandwith_params)
# Aは従来の患者データ。fit()することで上記の機械学習モデルをトレーニングしてデータ内のパターンを学習してもらう
grid_search.fit(A)
kde = grid_search.best_estimator_

# rand_stateの定義（rand_stateは新しいデータを作成する際のseedの定義）
rand_state = 11
# トレーニングしたモデルを使い新しい症例を20例作成してもらう
new_data = kde.sample(20, random_state = rand_state)

#　新しい症例を元のデータに加る
A_updated = np.concatenate((A, new_data), axis=0)

In [14]:
print(A_updated)
print(A_updated.shape)

[[ 0.00000000e+00  0.00000000e+00  7.80000000e+01 ...  3.20000000e+00
   3.20000000e+00  4.22195768e+00]
 [ 0.00000000e+00  1.00000000e+00  7.20000000e+01 ...  3.20000000e+00
   3.20000000e+00  2.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  8.60000000e+01 ...  3.20000000e+00
   3.20000000e+00  2.00000000e+00]
 ...
 [-1.68350206e-01  2.17062849e+00  7.54495993e+01 ...  4.41747449e+00
   3.43491011e+00  2.68344722e-01]
 [-9.01467055e-02  4.51755038e-01  8.50369950e+01 ...  6.59091640e-01
   2.87582520e+00  2.47272135e+00]
 [ 1.49239244e+00 -7.64538019e-01  7.86001193e+01 ...  2.28906569e+00
  -2.75123505e-02  1.22170628e+00]]
(80, 62)


In [15]:
# 行数と列数を変数として保管する
NUM_ROWS, NUM_COLS = A_updated.shape
print(f"\nこのデータセットの行数：{NUM_COLS}、列数：{NUM_ROWS}")


このデータセットの行数：62、列数：80


In [16]:
# データセットのフィーチャー・ラベル定義

X_all = A_updated[:,0:62].copy()  # X (フィーチャー) ... 全ての行
y_all = A_updated[:,0].copy()    # y (ラベル) ... 全ての行, 1列目のみ

# Create a copy of X_all and y_all to apply rf analysis to later
X_all_rf = X_all.copy()
y_all_rf = y_all.copy()

print(f"X_all (フィーチャーのみ: 10行まで) \n {X_all[:10,:]}")
print(f"y_all (ラベルのみ) \n {y_all}") 

X_all (フィーチャーのみ: 10行まで) 
 [[0.00000000e+00 0.00000000e+00 7.80000000e+01 1.00000000e+00
  1.60000000e+03 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 2.91666667e+02 3.45000000e+02 2.35000000e+00
  1.59900000e+03 4.10000000e+01 3.80000000e+01 3.00000000e+00
  1.30000000e+01 3.50000000e+00 6.64773204e+02 1.33137916e+04
  6.21768045e+01 4.04388091e+01 9.76562500e+00 4.45038295e+00
  6.24694689e+00 1.09631327e+01 3.20000000e+00 1.08365991e+01
  3.20000000e+00 3.20000000e+00 3.32659886e+01 3.20000000e+00
  3.20000000e+00 9.54746558e+02 3.20000000e+00 3.47345075e+01
  3.20000000e+00 8.15306905e+01 3.20000000e+00 3.20000000e+00
  3.20000000e+00 1.26679837e+01 3.20000000e+00 3.95799401e+02
  3.37974706e+00 3.20000000e+00 3.20000000e+00 3.20000000e+00
  3.20000000e+00 1.44513353e+01 3.20000000e+00 2.30161662e+01
  3.20000000e+00 3.13439925e+02 2.16094095e+03 1.00987865e+03
  2.89063159e+01 3.09405765e+01 4.48326515e+00 3.20000000e+00
  3.20000000e+00 4.22195768e+00]
 [0.0000000

In [17]:
# フィーチャーの重要性を表すpythonのディクショナリー
# 最初の値は全ての行に1（データの内容を変えないように１で掛け算してもデータは変わらない）で指定しておけば、RFモデルがトレーニングする際に自動で更新してくれる
COL_WEIGHT_DICT = {}

for colname in AE_FILD_df_idf.columns[1:]: # 最初の行が出力データを表すため、フィーチャーの重みは最初の行以外を考慮する
  i = AE_FILD_df_idf.columns.get_loc(colname)
  weight = 1
  COL_WEIGHT_DICT[colname] = 1
  print(f"行名{colname}の重みを{weight}に指定しました")
  X_all[:,i] *= weight #　そのコラム内のデータをweightで掛け算する

行名sex male:0 female:1の重みを1に指定しました
行名age の重みを1に指定しました
行名smoking status Never:0 Ex(quit smoking over a year ago):1 Current:2の重みを1に指定しました
行名Brinkman Indexの重みを1に指定しました
行名emphysema (-):0 (+):1の重みを1に指定しました
行名Baseline ILD UIP:0 probable UIP:1 Indeterminate for UIP:2 Alternative diagnosis:3の重みを1に指定しました
行名Collagen disease (-):0 (+):1の重みを1に指定しました
行名IPAF (-):0 (+):1の重みを1に指定しました
行名P/F ratioの重みを1に指定しました
行名LDHの重みを1に指定しました
行名CRPの重みを1に指定しました
行名KL-6の重みを1に指定しました
行名 BAL(neutro %)の重みを1に指定しました
行名 BAL(Lymph %)の重みを1に指定しました
行名 BAL(Eos %)の重みを1に指定しました
行名 BAL(M %)の重みを1に指定しました
行名 BAL(CD4/8)の重みを1に指定しました
行名MMP-1の重みを1に指定しました
行名MMP-7の重みを1に指定しました
行名TGFB1の重みを1に指定しました
行名TGFB2の重みを1に指定しました
行名TGFB3の重みを1に指定しました
行名EGFの重みを1に指定しました
行名FGF-2の重みを1に指定しました
行名EOTAXINの重みを1に指定しました
行名TGF-aの重みを1に指定しました
行名G-CSFの重みを1に指定しました
行名Flt-3Lの重みを1に指定しました
行名GM-CSFの重みを1に指定しました
行名FRACTALKINEの重みを1に指定しました
行名IFN-a2の重みを1に指定しました
行名IFN-gの重みを1に指定しました
行名GROの重みを1に指定しました
行名IL-10の重みを1に指定しました
行名MCP-3の重みを1に指定しました
行名IL-12P40の重みを1に指定しました
行名MDCの重みを1に指定しました
行名IL-12P70

In [18]:
# トレーニングデータとテストデータに入力データを分ける
# トレーニングデータ内のどのデータをどちらのデータセットに振り分けるかの指定をnpのrandom.permutationにより、毎回機械学習を行う際にランダムで決めてもらう
indices = np.random.permutation(len(y_all)) # ラベルの行のindexを記憶している変数
print(f"indices: {indices}")

# X_allとy_allに上記でnpが主力したランダムindexでデータセットの内容をシャッフルする （注：X_allとy_allは必ず同じindexでシャッフルしなくてはいけない）
X_all = X_all[indices]
y_all = y_all[indices]

print("置き換わったフィーチャーリスト：\n", X_all[:10,:])
print("置き換わったラベルリスト：\n", y_all)

indices: [52 48 10 12 13 59 33 29 56 22 21  7 17 37 23 54 57 78 40 77 79 18 32  6
 31 42 28 63 58 11 49 39 74 19 73 36 51  8 44 55 20  9 25 45  1 16 69  5
 15 34  4 35 71 76 46 70 66  0 30 38 60 27 61 68 67  3 26 64 41 14 50 43
 53 24 65 62 47  2 75 72]
置き換わったフィーチャーリスト：
 [[1.00000000e+00 0.00000000e+00 7.70000000e+01 2.00000000e+00
  2.85000000e+03 0.00000000e+00 3.00000000e+00 0.00000000e+00
  0.00000000e+00 1.42666667e+02 4.27000000e+02 1.78000000e+01
  1.13100000e+03 9.00000000e+00 1.00000000e+01 2.00000000e+00
  7.90000000e+01 2.30000000e+00 7.82108200e+02 1.19534998e+04
  9.76562500e+00 3.37680083e+01 9.76562500e+00 6.49592248e+00
  6.24694689e+00 1.56957789e+01 3.20000000e+00 5.25782261e+01
  3.20000000e+00 5.12747775e+00 5.83876303e+01 3.20000000e+00
  3.20000000e+00 1.15013084e+03 3.20000000e+00 2.31570024e+01
  3.20000000e+00 4.87607169e+01 3.20000000e+00 3.20000000e+00
  3.20000000e+00 3.20000000e+00 3.20000000e+00 2.57196104e+02
  3.20000000e+00 3.20000000e+00 3.20000000e+00

In [19]:
# トレニーングデータとテストデータの振り分け
# 割合：トレーニング（8割）・テスト（2割）
NUM_ROWS = X_all.shape[0] #　列数
TEST_PERCENT = 0.2 #　TEST ＋　TRAINING は必ず1にならなければいけない
TEST_SIZE = int(TEST_PERCENT * NUM_ROWS) 

X_test = X_all[:TEST_SIZE] # 0からTEST＿SIZEまでのデータ
y_test = y_all[:TEST_SIZE]

X_train = X_all[TEST_SIZE:] # TEST_SIZE以降のデータ
y_train = y_all[TEST_SIZE:]

print(f"トレーニング列数：{len(y_train)}、テスト列数：{len(y_test)}")

トレーニング列数：64、テスト列数：16


In [20]:
# RF解析に使うsklearnのライブラリーの導入
from sklearn import tree #　決定木モデル
from sklearn import ensemble # RF解析用のモデル
from sklearn.model_selection import cross_val_score # RF解析の最適解層をcross validationを使い見つけるためのモデル

In [21]:
# モデルの構築・モデルのトレーニング
# RF モデルの構築
# cross_valを使い、最適層を決めるんですが、事前に何階層を使えばいいかわからないため、適当に6階層決定木42と指定する
best_depth = 6
best_num_trees = 42
rforest_model = ensemble.RandomForestClassifier(max_depth=best_depth, n_estimators=best_num_trees) # ensemble.RandomForestClassifierからRF解析モデルを引っ張ってくる

# モデルの初期トレーニング
rforest_model.fit(X_train, y_train) # .fit()がトレーニングデータを使い、モデルをトレーニングしろというコマンド
print(f"RFモデルを{best_depth}層と{best_num_trees}決定木で作成しました")

ValueError: ignored

In [ ]:
#　初期トレーニング後のモデルのパフォーマンス検証
# テストデータにrforest_modelに.predict()関数を応用し、RFモデルがラベルを何個正確に予測出来たかを検証する
predicted_labels = rforest_model.predict(X_test)
actual_labels = y_test

# ラベルをプリントして実際に予測vs.実際のラベルを見てみる
print(f"予測ラベル：{predicted_labels}")
print(f"実際ラベル: {actual_labels}")

# 正確性の出力
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"正確性は：{num_correct/total * 100}％でした")

In [ ]:
# cross validationを使い、階層や決定木の数をチューニングしてみる（ハイパーパラメーター）
# 注：過適合/過剰適合のバランスを見極めながらハイパーパラメーターチューニングを行う
best_accuracy = 0
best_depth = 1
best_num_trees = 42

for ntrees in range(50,450,100):  # 決定木50から350の間（ステップ数：100)
    for d in range(1,20): # 階層1から20の間
        rforest_model = ensemble.RandomForestClassifier(max_depth=d, 
                                                        n_estimators=ntrees)
        cv_scores = cross_val_score( rforest_model, X_train, y_train, cv=5 ) # 5 はトレーニング・テストの割合が8割・2割ということを表す
        average_cv_accuracy = cv_scores.mean()  # cvスコアの平均値cv=5はスコアを五つ出すため、その平均値を見る
        if average_cv_accuracy >= best_accuracy: # cvスコアの平均値正確性が現在の最適正確性を上回った場合、現在のcvスコア平均値を出した決定木数・階層を最適とする
            best_accuracy = average_cv_accuracy
            best_depth = d
            best_num_trees = ntrees
        print(f"階層: {d:2d} 決定木: {ntrees:3d} cv スコア平均値: {average_cv_accuracy:7.4f}")

print()
print(f"最適層: {best_depth}, 最適決定木数： {best_num_trees}, 最高予測精度：{best_accuracy}")

In [ ]:
# 上記で見つけた最適層・最適決定木数をもとにモデルの構築・モデルのトレーニング
rforest_model = ensemble.RandomForestClassifier(max_depth=best_depth, n_estimators=best_num_trees) # ensemble.RandomForestClassifierからRF解析モデルを引っ張ってくる

# モデルを再度トレーニングする
rforest_model.fit(X_train, y_train) # .fit()がトレーニングデータを使い、モデルをトレーニングしろというコマンド
print(f"RFモデルを{best_depth}層と{best_num_trees}決定木で作成しました")

In [ ]:
#  ラベル毎に予測ラベルが実際のラベルと一致してるか確認する関数
def compare_labels(predicted_labels, actual_labels):
    NUM_LABELS = len(predicted_labels)
    num_correct = 0
    
    for i in range(NUM_LABELS): 
        p = int(round(predicted_labels[i]))         # round protects from fp error 
        a = int(round(actual_labels[i]))
        result = "incorrect"
        if p == a:  # if they match,
            result = ""       # no longer incorrect
            num_correct += 1  # and we count a match!

        print(f"row {i:>3d} : {p} {a}   {result}") 

    print(f"\n正確性は：{num_correct/NUM_LABELS * 100}％でした")
    return num_correct

In [ ]:
# 再度トレーニングした後のモデルのパフォーマンス検証
# テストデータにrforest_modelに.predict()関数を応用し、RFモデルがラベルを何個正確に予測出来たかを検証する
predicted_labels = rforest_model.predict(X_test)
actual_labels = y_test

# ラベルをプリントして実際に予測vs.実際のラベルを見てみる
print(f"予測ラベル：{predicted_labels}")
print(f"実際ラベル: {actual_labels}")

# 正確性の出力
compare_labels(predicted_labels, actual_labels)

In [ ]:
# フィーチャーの重要度を出力するコード
feature_importances = rforest_model.feature_importances_

feature_importances_dict = {}
for i, importance in enumerate(feature_importances):
  perc = importance * 100
  feature_importances_dict[AE_FILD_df_idf.columns[i]] = perc

sorted_dict = {}
sorted_keys = sorted(feature_importances_dict, key=feature_importances_dict.get)
for w in sorted_keys:
    sorted_dict[w] =  feature_importances_dict[w]
  
for keys in sorted_dict.keys():
  print(f"フィーチャー名：{keys:>12s}は出力データの結果に{sorted_dict[keys]:>7.2f}%関与しています")